In [2]:
import sys

# This to get the peerless target star DataFrame for example purposes
sys.path.append('/u/tdm/repositories/peerless/prediction')
sys.path.append('/u/tdm/repositories/peerless')
from targets import targets

# The action is here. Depends on vespa & isochrones.
from exosyspop.populations import KeplerBinaryPopulation

In [3]:
pop = KeplerBinaryPopulation(targets, fB=0.4)

In [4]:
# Accessing secondary properties will initialize a secondary simulation,
# calling pop._generate_binaries().  The first time this is called, the
# secondary property regressors get trained.
pop.radius_B

dmag regressor trained, R2=0.999564514011
qR regressor trained, R2=0.999534316094


array([ 0.26559583,  0.7196393 ,         nan, ...,  0.56702034,
               nan,         nan])

In [5]:
# subsequent calls are much faster; e.g.
pop._generate_binaries()
print(pop.radius_B)
%timeit pop._generate_binaries()

[ 0.82289165         nan         nan ...,         nan  0.2560088
         nan]
10 loops, best of 3: 89.7 ms per loop


In [6]:
# If physical accuracy is important, you can also choose to generate binary properties
# directly from the isochrone, but it's a factor of a few slower:
pop._generate_binaries(use_ic=True)
print(pop.radius_B)
%timeit pop._generate_binaries(use_ic=True)

[ 0.60007966         nan         nan ...,         nan         nan
         nan]
1 loops, best of 3: 350 ms per loop


In [7]:
# Similarly, accessing orbital properties will generate them
pop.period

array([  1.34150430e+07,   6.93327381e+02,   1.20863825e+07, ...,
         9.21835330e+00,   1.98839765e+06,   8.59759654e+01])

# Synthetic observations

In [8]:
# Now, we can observe and see what we see.  This takes into account
# duty cycle & data span, as well as geometry.
obs = pop.observe()
print(len(obs))
print(obs.columns)
obs.head()

508
Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'phase_sec'],
      dtype='object')


,index,period,ecc,w,inc,a,aR,b_pri,b_sec,k,...,T14_pri,T14_sec,T23_pri,T23_sec,dataspan,dutycycle,flux_ratio,n_pri,n_sec,phase_sec
0,567,2.329654,0.121586,0.681406,1.439450,6.210832e+11,8.864532,1.062451,1.238684,0.654890,...,0.099055,0.099797,0.000000,0.000000,1459.789,0.6988,0.109062,446,425,0.560206
1,1024,6.184446,0.055348,6.243595,1.535623,1.111991e+12,15.805362,0.555328,0.552900,0.373070,...,0.156771,0.156219,0.036287,0.036698,1459.789,0.6988,0.007201,154,153,0.535190
2,1137,43.819884,0.546291,5.575145,1.560137,3.816043e+12,84.627923,0.981634,0.466974,0.694193,...,0.295701,0.165909,0.000000,0.000000,1459.789,0.6988,0.150103,21,27,0.773045
3,1275,1.361283,0.725103,5.322146,0.975177,3.756096e+11,7.600412,4.985818,1.268227,0.533725,...,0.000000,0.025737,0.000000,0.000000,1459.789,0.8752,0.038834,0,938,0.813478
4,1639,1.524931,0.458634,0.785781,1.183417,4.055987e+11,5.017056,1.129998,2.215306,0.132489,...,0.005263,0.000000,0.000000,0.000000,1459.789,0.8749,0.000560,831,0,0.713830


In [9]:
# This is pretty fast, even when generating a new population each time:
%timeit pop.observe(new=True)

1 loops, best of 3: 180 ms per loop


In [10]:
# Even faster if we only generate new orbits.
%timeit pop.observe(new_orbits=True)

10 loops, best of 3: 92.8 ms per loop


In [11]:
# So we can predict the expected number of observations pretty easily.
import numpy as np
N = 100
n_obs = np.array([len(pop.observe(new_orbits=True)) for i in range(N)])
n_obs.mean(), n_obs.std()

(525.24000000000001, 21.941795733257567)

In [11]:
# Notice that the above does not yet have trapezoidal parameters.  There are two options to generate these.
# Either we can set the fit_trap parameter, as follows:
obs = pop.observe(fit_trap=True)
print(len(obs))
obs.columns

545


Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'phase_sec', u'trap_dur_pri',
       u'trap_depth_pri', u'trap_slope_pri', u'trap_dur_sec',
       u'trap_depth_sec', u'trap_slope_sec'],
      dtype='object')

In [12]:
# All things considered, this is still pretty fast if we just need to do it a few times:
%timeit pop.observe(fit_trap=True)

1 loops, best of 3: 2.51 s per loop


In [13]:
# However, this is pretty slow if we want to do inference.  To help with this, we can 
# tell it to train & use a regression.  Training only happens once; by default with 10,000 
# synthetic observations.  This takes a minute or so.
obs = pop.observe(regr_trap=True)
print(len(obs))
obs.columns

Depth trained: R2=0.997913681986
Duration trained: R2=0.999125594333
Slope trained: R2=0.998668293268
534


Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'phase_sec', u'trap_dur_pri_regr',
       u'trap_depth_pri_regr', u'trap_slope_pri_regr', u'trap_dur_sec_regr',
       u'trap_depth_sec_regr', u'trap_slope_sec_regr'],
      dtype='object')

In [14]:
# Subsequent calls are much faster
%timeit pop.observe(regr_trap=True)

10 loops, best of 3: 33 ms per loop


In [15]:
# Even generating a new stellar population & observing it is pretty quick
%timeit pop.observe(regr_trap=True, new=True)

1 loops, best of 3: 200 ms per loop


In [16]:
# Or again, you can just generate new orbits (rather than new binaries & new orbits)
%timeit pop.observe(regr_trap=True, new_orbits=True)

10 loops, best of 3: 114 ms per loop


In [17]:
# Generating the training data used for the trapezoid shape regression above used
# this function, which can be otherwise useful to sample >N random observations 
# from the existing population.  `trap_regr` defaults to `True` here.  
# This function also takes `new` or `new_orbits` keywords.
obs_pop = pop.get_N_observed(N=10000, new_orbits=True)
print(len(obs_pop))
obs_pop.columns

10441


Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'phase_sec', u'trap_dur_pri_regr',
       u'trap_depth_pri_regr', u'trap_slope_pri_regr', u'trap_dur_sec_regr',
       u'trap_depth_sec_regr', u'trap_slope_sec_regr'],
      dtype='object')

In [18]:
# We can now look, e.g. at the expected number of single/double eclipsing systems:
query = '(n_pri < 3) & (n_sec < 3) & (n_pri==0 | n_sec==0)'
N = 100
n_obs = np.array([len(pop.observe(new_orbits=True).query(query)) for i in range(N)])
n_obs.mean(), n_obs.std()

(6.1299999999999999, 2.4684205476376992)

In [19]:
# Try this again, this time using the empirical eccentricity distribution
# (as opposed to the beta distribution with default params)---eccentricity matters!
pop.ecc_empirical = True
n_obs = np.array([len(pop.observe(new_orbits=True).query(query)) for i in range(N)])
n_obs.mean(), n_obs.std()

(10.210000000000001, 3.2780939583849635)